### Atividade 1 — Estatísticas por Categoria

Dado o seguinte dataset: <br>
dados = {
'Categoria': ['A', 'B', 'A', 'C', 'B', 'C', 'A'],
'Valor': [10, 20, 15, 30, 25, 35, 12]
}
<br>
Objetivo: Com Pandas e de seguida com Polars, calcular:<br>
-Total de valores por categoria<br>
-Média por categoria<br>
-Desvio padrão<br>

In [2]:
## carregamento de Bibliotecas

import pandas as pd
import polars as pl

dados = {
'Categoria': ['A', 'B', 'A', 'C', 'B', 'C', 'A'],
'Valor': [10, 20, 15, 30, 25, 35, 12]
}


####  Com Pandas

In [ ]:

df= pd.DataFrame(dados).reset_index()
print ('Contagem de nulos')
print(df.isnull().sum()) #verifica se existem valores nulos
#Calculos individuais
Total=df.groupby('Categoria')['Valor'].sum()
Media=df.groupby('Categoria')['Valor'].mean()
Desvio=df.groupby('Categoria')['Valor'].std()
print('\nTotal por Categoria:', Total)
print('\nMedia por Categoria:', Media) 
print('\nDesvio Padrão por Categoria:', Desvio)

#Passagem de dados para tabela
print('\nTabela com dados ')
tabela = pd.DataFrame({'Total': Total, 'Media': Media, 'Desvio': Desvio})
print(tabela)

print('\n Resumo de dados agregados com Pandas')
agregado = df.groupby('Categoria')['Valor'].agg(['count', 'mean', 'std', 'min', 'max'])
print(agregado) 

####  Com Polars

In [ ]:
df = pl.DataFrame(dados) 

# Resumo de dados com Polars
tabela=df.group_by('Categoria').agg(
    [   pl.col('Valor').count().alias('Count'),  
        pl.col('Valor').sum().alias('Total'),
        pl.col('Valor').mean().alias('Media'),
        pl.col('Valor').std().alias('Desvio'),
        pl.col('Valor').min().alias('Min'),
        pl.col('Valor').max().alias('Max')
    ]
).sort('Categoria')

print(tabela)


### Actividade 2

#### 1. Usando Pandas, crie um DataFrame com notas de alunos por disciplina. Agrupe por disciplina e calcule a média e o desvio padrão das notas.

In [ ]:
alunos = {
        'Disciplinas':['Matematica', 'Portugues', 'Historia', 'Ingles','Portugues', 'Historia', 'Ingles'],
        'Notas':[10,9.0,11.0,12,8.5,11,9.0]
        }

df_alunos = pd.DataFrame(alunos).reset_index()
print('\nContagem de nulos')
print(df_alunos.isnull().sum()) #verifica se existem valores nulos

print('\nTabela de Alunos')
print(df_alunos)

print('\n Resumo de dados agregados com Pandas')
agregado = df_alunos.groupby('Disciplinas')['Notas'].agg(['count', 'mean', 'std', 'min', 'max'])
print(agregado) 



#### 2. Com Polars, carregue um CSV com dados de vendas e agrupe por região, calculando o total e o número de vendas por região

In [ ]:
df_vendas = pl.read_csv(r'C:\Users\HP\Desktop\Formação\Eisnt\UFCD 10808 - Limpeza e transformação de dados em Python\vendas.csv')

print('\nTabela de Vendas')
print(df_vendas)
# Verificar se existem valores nulos    
print('\nContagem de nulos')
print(df_vendas.select(pl.all().is_null().sum())) #verifica se existem valores nulos


# Agrupar por região e calcular total e número de vendas
resultado = df_vendas.group_by('regiao').agg(
    [
        pl.sum("valor_venda").alias("total_vendas"),
        pl.count("id_venda").alias("numero_vendas")
    ]
    ).sort('regiao')

# Exibir resultados
print(resultado)


#### 3. Extra: Faça uma agregação com múltiplas colunas num mesmo groupby

In [81]:
data = {
    'disciplina': ['Matemática', 'Matemática', 'Química','Física', 'Física', 'Química', 'Química','Matemática', 'Matemática', 'Física', 'Física', 'Química', 'Química'],
    'nota': [8.5, 7.0, 10, 9.0, 6.5, 7.5, 8.0,11.0, 18.0, 15 , 9.5, 11.5, 10.5],
    'ano': [2021, 2020, 2020, 2021, 2020, 2021, 2021,2020 , 2021, 2020, 2021, 2020, 2021],
}

df = pl.DataFrame(data)
# print('\nTabela de Alunos')
# print(df)

print('\nContagem de nulos')
print(df.select(pl.all().is_null().sum())) 

# Agrupando por disciplina e ano, e calculando média e desvio padrão
result = df.group_by(['ano', 'disciplina']).agg([
    pl.col('nota').mean().alias('media_nota'),
    pl.col('nota').std().alias('desvio_padrao_nota')
]).sort('ano', 'disciplina')

#Adiciona coluna com classificação atraves da media da nota
result = result.with_columns([
pl.when(pl.col("media_nota") < 10).then(pl.lit("Mau"))
.when(pl.col("media_nota") < 13).then(pl.lit("Suficiente"))
.otherwise(pl.lit("Bom"))
.alias("classificacao")
])

print(result)



Contagem de nulos
shape: (1, 3)
┌────────────┬──────┬─────┐
│ disciplina ┆ nota ┆ ano │
│ ---        ┆ ---  ┆ --- │
│ u32        ┆ u32  ┆ u32 │
╞════════════╪══════╪═════╡
│ 0          ┆ 0    ┆ 0   │
└────────────┴──────┴─────┘
shape: (6, 5)
┌──────┬────────────┬────────────┬────────────────────┬───────────────┐
│ ano  ┆ disciplina ┆ media_nota ┆ desvio_padrao_nota ┆ classificacao │
│ ---  ┆ ---        ┆ ---        ┆ ---                ┆ ---           │
│ i64  ┆ str        ┆ f64        ┆ f64                ┆ str           │
╞══════╪════════════╪════════════╪════════════════════╪═══════════════╡
│ 2020 ┆ Física     ┆ 10.75      ┆ 6.010408           ┆ Suficiente    │
│ 2020 ┆ Matemática ┆ 9.0        ┆ 2.828427           ┆ Mau           │
│ 2020 ┆ Química    ┆ 10.75      ┆ 1.06066            ┆ Suficiente    │
│ 2021 ┆ Física     ┆ 9.25       ┆ 0.353553           ┆ Mau           │
│ 2021 ┆ Matemática ┆ 13.25      ┆ 6.717514           ┆ Bom           │
│ 2021 ┆ Química    ┆ 8.666667   ┆ 1.